### Tensorflow 
---

1. 目的 : 构建 `CNN` 网络，实现对数据集 `StanfordDogs` 的狗的品种预测，查看预测的准确性
2. CNN 网络架构 : 使用 `AlexNet` 实现，随着网络的深度加大，高度和宽度会减小，但是深度(通道)会增加
3. 数据集说明
    1. 包含有 120 个狗的种类的不同的图片
    2. 80% 用作训练， 20% 用作测试
    3. 数据集中的图片的**格式不同，尺寸不同**

In [1]:
# 找打文件的路径
path = '/home/lantian/Downloads/nloads/StanfordDog/Images/'

#### 预处理和TFRecord
---
1. 图像的尺寸不同，图像的类型不同所以我们需要采用不同的方式预处理成相同类型的图片然后转化成 `TFRecord` 文件中去处理
2. 预处理和对转换类型有助于加速训练步骤，但是处理的时间相对来说也会比较长
3. 对于 `lazy loading` 导致计算图变得非常巨大的时候，使用 `sess.run` 的速度将会越来越慢，这是因为计算图没有变成只读导致计算图不断的变化变大引起的(计算图变大但是只要不执行 `sess.run` 并没有什么影响，就是内存吃的比较多)，就像下面的例子中我们不能再创建 `TFRecord` 文件的循环中加入 `sess.run` 的函数执行获得张量，相反的，我们可以考虑采用其他的图形处理库 `PIL` 将图片数据解析将数据保存在 `TFRecord` 文件中
4. `PIL` 写入 `TFRecord` 文件的注意点
   * 写入的时候，直接执行 `Image.tobytes()` 将图片数据直接处理成二进制文件文件(每一个像素数据都是 uint8 类型的)，然后直接写入 `TFReocrd` 文件
   * 读取的时候，使用方法 `tf.decode_raw(img_raw, tf.uint8)` 直接读取
   * 因为 `PIL` 中的数据存储结构是 `(width, height)` 类型，和我们的写入之前 `reshape` 的结构是刚好相反的(`(height, width)`),这里需要注意反过来对图片张量 `reshape`
   * 恢复 `label` 的时候，需要使用 `tf.cast(label, tf.string)` 的方式从张量恢复成二进制字符串，之后再用 `decode` 解码成字符串

In [ ]:
# TFRecord 文件的读写最佳实践

import tensorflow as tf
from PIL import Image
import numpy as np

# 写

'''
img = Image.open('test.png')
img = img.convert('L')
img = img.resize((250, 151))
img = img.tobytes()

writer = tf.python_io.TFRecordWriter('test.tfrecord')

label = 'test'.encode('utf8')

example = tf.train.Example(features = tf.train.Features(feature = {
        'label' : tf.train.Feature(bytes_list = tf.train.BytesList(value=[label])),
        'image' : tf.train.Feature(bytes_list = tf.train.BytesList(value=[img]))
    }))
writer.write(example.SerializeToString())
writer.close()
'''

# 读

fq = tf.train.string_input_producer(['test.tfrecord'])

reader = tf.TFRecordReader()
_, ser = reader.read(fq)
sess = tf.Session()
sess.run(tf.local_variables_initializer())
coord =  tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess = sess, coord = coord)

features = tf.parse_single_example(ser, features = {
    'label' : tf.FixedLenFeature([], tf.string),
    'image' : tf.FixedLenFeature([], tf.string)
    })

img = tf.decode_raw(features['image'], tf.uint8)
img = sess.run(img)
img = img.reshape((151, 250))
img = Image.fromarray(img)
img.show()
# label 的恢复处理方式
label = tf.cast(features['label'], tf.string)
print(sess.run(label).decode('utf8'))

In [ ]:
#!/usr/bin/python3

import tensorflow as tf
import glob
from PIL import Image
import numpy as np

# 收集数据
from itertools import groupby
from collections import defaultdict
import time

# 编写 TFRecord 文件, 一个文件写入 100 个数据文件及其标签，节省I/O读写次数，这样的做法可以提前的确定一个 Batch 但是不能随机的构建 batch
def writer_records_file(dataset, path):
    '''
    1. dataset - defaultdict 的一个键值对
    2. path    - 指定的保存路径
    '''
    
    writer = None
    current_index = 0
    sess = tf.Session()
    # sess.graph.finalize()
    graph = tf.get_default_graph()

    for breed, images_filenames in dataset.items():
        # 种类和对应的文件名队列
        print(breed, current_index)
        for image_filename in images_filenames:
            if current_index % 100 == 0:
                if current_index != 0 :
                    end = time.time()
                if writer:
                    # 存在读写器关闭
                    writer.close()
                
                # 不存在读写器创建
                record_filename = '%s/%s.tfrecords' % (path, str(current_index))
                print(record_filename)
                writer = tf.python_io.TFRecordWriter(record_filename)
            current_index += 1    
            time_photo = time.time()
            image_file = tf.read_file(image_filename)
            
            image_file = Image.open(image_filename)
            image_file = image_file.convert('L')
            image_file = image_file.resize((250, 151))
            img_data = image_file.tobytes()
            '''
            # 不是 jpeg 文件可以忽略不处理
            try:
                image = tf.image.decode_jpeg(image_file)
            except:
                print(image_filename)
                continue
            
            # 转换成灰度图片，加快处理速度
            # gray_image = tf.image.rgb_to_grayscale(image)
            # resized_gray_image = tf.image.resize_images(gray_image, [250, 151], method = 0)
            # print(sess.run(resized_gray_image).shape)
            # 转换成整数之后转变成字节数组
            pp = tf.cast(resized_gray_image, tf.uint8)
            pause = sess.run(pp)
            '''
            # the default graph become bigger and bigger, lazy loading problem find in CS20, The trap of lazy loading
            # 使用PIL读取图片送入计算图中
            # image_bytes = pause.tobytes()
            
            image_label = breed.encode('utf8')
            example = tf.train.Example(features = tf.train.Features(feature = {
                'label' : tf.train.Feature(bytes_list = tf.train.BytesList(value=[image_label])),
                'image' : tf.train.Feature(bytes_list = tf.train.BytesList(value=[img_data]))
            }))
            
            writer.write(example.SerializeToString())
    writer.close()


# 加载 TFRecord 文件
def read_tfrecord(path):    
    sess = tf.Session()
    
    filename_queue = tf.train.string_input_producer(glob.glob(path + 'testing-image/*'))
    reader = tf.TFRecordReader()
    _, ser = reader.read(filename_queue)
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess = sess, coord = coord)
    
    features = tf.parse_single_example(ser, features = {
        'label' : tf.FixedLenFeature([], tf.string),
        'image' : tf.FixedLenFeature([], tf.string)
    })
    
    record_image = tf.decode_raw(features['image'], tf.uint8) 
    # 灰度图像
    # image = tf.reshape(record_image, [250, 151, 1])

    p = sess.run(record_image)
    image = p.reshape((151, 250))
    label = tf.cast(features['label'], tf.string)
    
    # create the shuffle_batch
    min_after_dequeue = 10
    batch_size = 3
    capacity = min_after_dequeue + 3 * batch_size
    # 构建样本队列

    image_batch, label_batch = tf.train.shuffle_batch([image, label], batch_size = batch_size, capacity=capacity, min_after_dequeue=min_after_dequeue)
    print('before ... ')
    return image_batch, label_batch

if __name__ == "__main__":
    # 找打文件的路径
    path = '/home/lantian/Downloads/StanfordDog/Images/'

    # get the label and the image
    coord = tf.train.Coordinator()
    try:
        image_batch, label_batch = read_tfrecord(path)
        sess = tf.Session()
        threads = tf.train.start_queue_runners(sess=sess, coord = coord)
        img, lab = sess.run([image_batch, label_batch])
    except tf.errors.OutOfRangeError:
        print('Over the reading ... ')
    finally:
        coord.request_stop()
    coord.join(threads)
    sess.close()

    '''
    # 提取所有的图片
    test_number = 0
    train_number = 0
    image_filename = glob.glob(path + 'n02*/*.jpg')
    training = defaultdict(list)
    testing = defaultdict(list)
    # filename.split('/')[-2] 抽取的是狗的种类
    image_filename_with_breed = map(lambda filename: (filename.split('/')[-2], filename), image_filename)
    
    for dog_breed, breed_images in groupby(image_filename_with_breed, lambda x : x[0]):
        
        # 20 % 的数据加入训练集
        for i, breed_image in enumerate(breed_images):
            if i % 5 == 0:
                testing[dog_breed].append(breed_image[1])
                test_number += 1
            else:
                training[dog_breed].append(breed_image[1])
                train_number += 1
            
        # 确保测试集的数目 > 18 % 
        breed_training_count = len(training[dog_breed])
        breed_testing_count = len(testing[dog_breed])
        
        assert round(breed_testing_count / breed_training_count, 2) > 0.18, "Do not have the enough test dataset !"

    writer_records_file(testing, path + 'testing-image')
    print('testing is over!')
    writer_records_file(training, path + 'training-image')
    print('trainging is over!')
    '''

In [ ]:
# Model 模型构建
batch_size = 3

# 转换成浮点类型 [0, 1] 更适合卷积处理
float_image_batch = tf.image.convert_image_dtype(image_batch, tf.float32)

# 第一层卷积和池化
conv2d_layer_1 = tf.contrib.layers.convolution2d(float_image_batch, num_outputs=32, kernel_size=(5, 5),
                                                activation_fn = tf.nn.relu, weight_initializer = tf.random_normal,
                                                stride = (2, 2), trainable= True)
pool_layer_1 = tf.nn.max_pool(conv2d_layer_1, ksize=[1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

# 第二层卷积和池化
conv2d_layer_2 = tf.contrib.layers.convolution2d(pool_layer_1, num_outputs=64, kernel_size=(5, 5),
                                                activation_fn = tf.nn.relu, weight_initializer = tf.random_normal,
                                                stride = (1, 1), trainable= True)
pool_layer_2 = tf.nn.max_pool(conv2d_layer_1, ksize=[1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

# 全连接
flattern_layer_2 = tf.reshape(pool_layer_2, [batch_size, -1])
hidden_layer_3 = tf.nn.dropout(tf.contrib.layers.fully_connected(flattern_layer_2, 512), 0.2)

# 120 个狗的种类
final_layer_4 = tf.contrib.layers.fully_connected(hidden_layer_3, 120)